In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
from glob import glob
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from pandasql import sqldf
import random
import os


# 경고 끄기
warnings.filterwarnings(action='ignore')

# 시드고정
tf.random.set_seed(19970119)
random.seed(19970119)
np.random.seed(19970119)

In [ ]:

def add_dosomae(dir, option=1):

        """
        check = 중간 산출물을 저장하고 싶다면 check 을 0 이외의 숫자로
        domae, somae 데이터를 가져와서 정제하는 단계
        option parameter을 통한 도매, 소매 선택
        """
        data_list = glob(dir) # train raw 데이터 넣을지 / test raw 데이터 넣을지 경로
        domae = []
        somae = []

        for i in data_list:
            if 'domae' in i:
                domae.append(i)
            if 'somae' in i:
                somae.append(i)

        sum_df = pd.DataFrame()
                
        if option == 1:
            df = domae
            text = '도매'
        else:
            df = somae
            text = '소매'

        for i in tqdm(df):
            test = pd.read_csv(i)
            test.fillna(0,inplace=True) # 널값 0으로 채워주고 
            name = i.split('/')[-1].split('.')[0].split('_')[1]

            sep = test.loc[(test['등급명'] == '상품') | (test['등급명'] == 'S과') | (test['등급명'] == 0)]   # 모든 상품에 대해서 수행하지 않고 GRAD_NM이 '상품', 'S과' 만 해당하는 품목 가져옴
            sep = sep[['datadate', '등급명', '조사단위(kg)', '가격(원)']]
            
            
            sep['품목'] = [int(name)]*len(sep) # 품목 변수 만들어주기

            sep.rename(columns={"가격(원)": "가격"}, inplace=True)

            sep2 = sqldf(
                f"select datadate, 품목, max(가격) as '일자별_{text}가격_최대(원)', avg(가격) as '일자별_{text}가격_평균(원)', min(가격) as '일자별_{text}가격_최소(원)' from sep group by datadate")
            

            
            
            
            
            
            # globals()[f'df_{name.split("_")[1].split(".")[0]}'] = globals()[f'df_{name.split("_")[1].split(".")[0]}'].merge(sep2, how='left')

            # # 중간 산출물 저장
            # if check != 0:
            #     if os.path.exists(f'./data') == False:
            #         os.mkdir(f'./data')

            #     if os.path.exists(f'./data/{text}') == False:
            #         os.mkdir(f'./data/{text}')

            sum_df = pd.concat([sum_df, sep2])
        return sum_df 

In [ ]:
domae = add_dosomae('/content/drive/MyDrive/농산물예측/aT_data/aT_train_raw/domae*', 1)
somae = add_dosomae('/content/drive/MyDrive/농산물예측/aT_data/aT_train_raw/*', 2)

100%|██████████| 37/37 [00:25<00:00,  1.43it/s]


In [ ]:
domae.shape

(54057, 5)

In [ ]:
somae.shape

(52524, 5)

In [ ]:
somae

,datadate,품목,일자별_소매가격_최대(원),일자별_소매가격_평균(원),일자별_소매가격_최소(원)
0,20130101,0,0.0,0.000000,0.0
1,20130102,0,5980.0,4298.522727,3210.0
2,20130103,0,5980.0,4298.522727,3210.0
3,20130104,0,5980.0,4298.522727,3210.0
4,20130105,0,5980.0,4298.522727,3210.0
...,...,...,...,...,...
1456,20161227,6,0.0,0.000000,0.0
1457,20161228,6,0.0,0.000000,0.0
1458,20161229,6,0.0,0.000000,0.0
1459,20161230,6,0.0,0.000000,0.0


In [ ]:
domae

,datadate,품목,일자별_도매가격_최대(원),일자별_도매가격_평균(원),일자별_도매가격_최소(원)
0,20130101,3,0.0,0.0,0.0
1,20130102,3,43000.0,39200.0,35000.0
2,20130103,3,43000.0,39200.0,35000.0
3,20130104,3,43000.0,39200.0,35000.0
4,20130105,3,43000.0,39600.0,35000.0
...,...,...,...,...,...
1456,20161227,7,0.0,0.0,0.0
1457,20161228,7,47800.0,35960.0,31000.0
1458,20161229,7,47800.0,37160.0,34000.0
1459,20161230,7,47800.0,37160.0,34000.0


In [ ]:
# concat dosomae
# 도매쪽으로 left join
def concat_dosomae(domae, somae):
  # merge를 위해 임의의 변수 생성 
  domae['merge_tmp1'] = domae.apply(lambda x: str(x['datadate'])+str(x['품목']), axis=1)
  somae['merge_tmp2'] = somae.apply(lambda x: str(x['datadate'])+str(x['품목']), axis=1)

  df = pd.merge(domae, somae, how='left', left_on='merge_tmp1', right_on='merge_tmp2')
  df.drop(['merge_tmp1', 'merge_tmp2', 'datadate_y', '품목_y'],axis=1,inplace=True)
  
  df.columns = ['datadate', '품목', '일자별_도매가격_최대(원)', '일자별_도매가격_평균(원)',
       '일자별_도매가격_최소(원)', '일자별_소매가격_최대(원)', '일자별_소매가격_평균(원)', '일자별_소매가격_최소(원)']
  

  df.fillna(0, inplace=True)
  return df



In [ ]:
df = concat_dosomae(domae, somae)
df

,datadate,품목,일자별_도매가격_최대(원),일자별_도매가격_평균(원),일자별_도매가격_최소(원),일자별_소매가격_최대(원),일자별_소매가격_평균(원),일자별_소매가격_최소(원)
0,20130101,3,0.0,0.0,0.0,0.0,0.000000,0.0
1,20130102,3,43000.0,39200.0,35000.0,30000.0,18206.052632,9900.0
2,20130103,3,43000.0,39200.0,35000.0,30000.0,18250.789474,9900.0
3,20130104,3,43000.0,39200.0,35000.0,30000.0,18250.789474,9900.0
4,20130105,3,43000.0,39600.0,35000.0,30000.0,18479.444444,12250.0
...,...,...,...,...,...,...,...,...
54052,20161227,7,0.0,0.0,0.0,0.0,0.000000,0.0
54053,20161228,7,47800.0,35960.0,31000.0,17980.0,12375.806452,8000.0
54054,20161229,7,47800.0,37160.0,34000.0,17980.0,12375.806452,8000.0
54055,20161230,7,47800.0,37160.0,34000.0,17980.0,12201.000000,8000.0


In [ ]:
df.columns

Index(['datadate', '품목', '일자별_도매가격_최대(원)', '일자별_도매가격_평균(원)', '일자별_도매가격_최소(원)',
       '일자별_소매가격_최대(원)', '일자별_소매가격_평균(원)', '일자별_소매가격_최소(원)'],
      dtype='object')

In [ ]:
df.isna().sum()

datadate          0
품목                0
일자별_도매가격_최대(원)    0
일자별_도매가격_평균(원)    0
일자별_도매가격_최소(원)    0
일자별_소매가격_최대(원)    0
일자별_소매가격_평균(원)    0
일자별_소매가격_최소(원)    0
dtype: int64

In [ ]:
!dir

drive  final_train_dosomae.csv	sample_data


In [ ]:
df.to_csv('final_train_dosomae.csv', index=False)

In [ ]:
somae.head(50)

,datadate,일자별_소매가격_최대(원),일자별_소매가격_평균(원),일자별_소매가격_최소(원)
0,20130101,0.0,0.000000,0.0
1,20130102,5980.0,4298.522727,3210.0
2,20130103,5980.0,4298.522727,3210.0
3,20130104,5980.0,4298.522727,3210.0
4,20130105,5980.0,4298.522727,3210.0
5,20130106,5980.0,4286.931818,3210.0
6,20130107,0.0,0.000000,0.0
7,20130108,0.0,0.000000,0.0
8,20130109,5980.0,4286.931818,3210.0
9,20130110,5980.0,4290.227273,3210.0
